In [ ]:
import importlib

packages = ["transformers", "datasets", "torch", "torchvision", "PIL", "accelerate"]

for package in packages:
    try:
        importlib.import_module(package)
        print(f"{package} is installed ✅")
    except ImportError:
        print(f"{package} is NOT installed ❌")


In [ ]:
import os

# Function to check if a path exists
def check_path_exists(path):
    if os.path.exists(path):
        print(f"The path '{path}' exists.")
    else:
        print(f"The path '{path}' does not exist.")

# Example usage
path_to_check = "color_img"
check_path_exists(path_to_check)
path_to_check = "pixel_values"
check_path_exists(path_to_check)
path_to_check = "color_images_metadata.json"
check_path_exists(path_to_check)

In [1]:
# Import Libraries and Configuration

import json
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoProcessor, AutoTokenizer, AutoModelForImageTextToText, Trainer, TrainingArguments
from PIL import Image
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"  

# === Fixing image tensors should be of type torch.float32 ===

folder_path = 'pixel_values'

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as file:
            pixel_values = np.array(json.load(file))
            pixel_values = torch.tensor(pixel_values).float()  # <-- Convert to float32 here
            
        print(f"🧩 {filename} - pixel_values shape: {pixel_values.shape}, dtype: {pixel_values.dtype}")

# === Load JSON Data ===
with open('color_images_metadata.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print("✅ JSON data loaded successfully!")

# === Initialize Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
print("✅ Tokenizer initialized successfully!")

# === Tokenize All Labels and Combine Into Tensor ===
labels_list = []  # Store all tokenized labels

# Loop through all entries in the JSON
for item in data:
    description = item["labels"]["description"]
    qa_pairs = " ".join([f"{qa['question']} {qa['answer']}" for qa in item["labels"]["QA_pairs"]])
    labels_text = f"Description: {description} QA: {qa_pairs}"

    # Tokenize each label with max_length and padding
    label = tokenizer(
        labels_text, 
        return_tensors="pt", 
        padding="max_length",    # ✅ Ensures all sequences have the same length
        truncation=True,
        max_length=30            # ✅ Adjust based on the longest sequence
    )["input_ids"]
    labels_list.append(label)

# Stack all labels into a single tensor
labels = torch.cat(labels_list, dim=0)

print("✅ All labels tokenized and combined into tensor!")


# === CONFIGURATION VARIABLES ===
# Remember to use the updated paths. I messed up the size, generating 214 instead of 224...
MODEL_NAME = "Qwen/Qwen2.5-VL-3B-Instruct"
JSON_PATH = "color_images_metadata.json"  
IMAGE_DIR = "color_img"                  
PIXEL_VALUES_DIR = "pixel_values"         
OUTPUT_DIR = "./qwen2.5_vl_finetuned"
LOG_DIR = "./logs"
BATCH_SIZE = 1
LEARNING_RATE = 5e-5
EPOCHS = 1
SAVE_LIMIT = 2
USE_FP16 = True
PUSH_TO_HUB = False
SEED = 42
TRAIN_SPLIT = 0.8

print("Libraries imported and configurations set")

🧩 pixel_values_74.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_98.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_4.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_67.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_96.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_38.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_14.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_18.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_93.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_2.json - pixel_values shape: torch.Size([3, 224, 224]), dtype: torch.float32
🧩 pixel_values_32.json - pixel_values shape: torch.Size([3, 224, 224]), 

In [2]:
# Cell 5: Debug Prints
print(f"Total labels loaded: {len(labels_list)}")
print(f"Labels tensor shape: {labels.shape}")
print(f"Labels dtype: {labels.dtype}")

# Cell 6: Print the First 5 Labels
for i in range(min(5, len(labels))):
    print(f"\n🔢 Entry {i + 1} - Labels as Token IDs:")
    print(labels[i])

    decoded_labels = tokenizer.decode(labels[i], skip_special_tokens=True)
    print("📝 Decoded Text:")
    print(decoded_labels)


Total labels loaded: 100
Labels tensor shape: torch.Size([100, 30])
Labels dtype: torch.int64

🔢 Entry 1 - Labels as Token IDs:
tensor([  5009,     25,   1096,    374,    264,   6437,   4158,   1894,   2168,
            13,  65908,     25,   2160,    419,   1894,   4158,     30,   7414,
          2160,    419,   1894,   1045,   1008,   1894,     30,   2308, 151643,
        151643, 151643, 151643])
📝 Decoded Text:
Description: This is a solid white color image. QA: Is this color white? Yes Is this color some other color? No

🔢 Entry 2 - Labels as Token IDs:
tensor([  5009,     25,   1096,    374,    264,   6437,   4878,  16576,   1894,
          2168,     13,  65908,     25,   2160,    419,   1894,   4878,  16576,
            30,   7414,   2160,    419,   1894,   1045,   1008,   1894,     30,
          2308, 151643, 151643])
📝 Decoded Text:
Description: This is a solid magenta color image. QA: Is this color magenta? Yes Is this color some other color? No

🔢 Entry 3 - Labels as Token IDs

In [3]:
print(f"pixel_values dtype: {pixel_values.dtype}")
print(f"labels dtype: {labels.dtype if 'labels' in locals() else 'No labels'}")

pixel_values dtype: torch.float32
labels dtype: torch.int64


In [4]:
# Set fixed Seed
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(SEED)

print("Seed has been set. Check the Configuration cell to adjust")

Seed has been set. Check the Configuration cell to adjust


In [5]:
import torch
from torch.utils.data import Dataset
import json
from PIL import Image

class ColorImageDataset(Dataset):
    def __init__(self, json_path, processor):
        print("🔄 Loading dataset...")
        with open(json_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)

        self.processor = processor
        self.qa_pairs = []

        # Use updated file paths
        for item in self.data:
            # Correct file paths
            img_path = item["file_link"].replace("model_input_test/color_img/", "color_img_resized/")
            pixel_values_path = item["pixel_values_link"].replace("model_input_test/pixel_values/", "pixel_values_resized/")
            
            # ✅ Ensure labels exist
            if "labels" not in item:
                print(f"❗ Warning: Missing 'labels' in item {item['index_num']}")
                continue

            # Access labels: description and QA pairs
            description = item["labels"]["description"]
            for qa in item["labels"]["QA_pairs"]:
                question = qa["question"]
                answer = qa["answer"]
                self.qa_pairs.append((img_path, pixel_values_path, description, question, answer))

        print(f"✅ Dataset loaded with {len(self.qa_pairs)} QA pairs.")

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        img_path, pixel_values_path, description, question, answer = self.qa_pairs[idx]

        # Debug prints to confirm paths
        print("🔍 Loading image from:", img_path)
        print("🔍 Loading pixel values from:", pixel_values_path)

        # Load the image
        image = Image.open(img_path).convert("RGB")

        # Load pixel values
        with open(pixel_values_path, 'r') as f:
            pixel_values = json.load(f)

        # ✅ Debug: Print description and question
        print("📝 Description:", description)
        print("❓ Question:", question)
        print("💬 Answer:", answer)

        # Encode input using Qwen processor
        inputs = self.processor(
            images=image,
            text=[f"Image Description: {description}\nQuestion: {question}"],
            return_tensors="pt",
            padding=True,
            truncation=True
        )

        # ✅ Debug: Check processor output
        print("📦 Processor output keys:", inputs.keys())

        # Tokenize the expected answer
        labels = self.processor.tokenizer(
            answer, 
            return_tensors="pt",
            padding=True,
            truncation=True
        )

        # ✅ Ensure labels exist and are within range
        vocab_size = self.processor.tokenizer.vocab_size
        if "input_ids" in labels and (labels["input_ids"] < vocab_size).all() and (labels["input_ids"] >= 0).all():
            inputs["labels"] = labels["input_ids"]
            print("🏷️ Final labels dtype:", inputs["labels"].dtype)
            print(f"🏷️ labels tensor shape: {inputs['labels'].shape}")
        else:
            print("❗ Warning: Labels out of range or missing 'input_ids'")

        # ✅ Convert pixel values to tensor
        pixel_tensor = torch.tensor(pixel_values, dtype=torch.float32)

        # ✅ Ensure correct shape for model input
        pixel_tensor = pixel_tensor.view(3, 224, 224)  # Standard RGB format
        pixel_tensor = pixel_tensor.unsqueeze(0)  # Ensure batch dimension (1, 3, 224, 224)
        print(f"📏 pixel_values tensor shape: {pixel_tensor.shape}")

        inputs["pixel_values"] = pixel_tensor

        # ✅ Add image_grid_thw required by Qwen2_5_VL
        inputs["image_grid_thw"] = torch.tensor([16, 16, 3], dtype=torch.int64)

        # ✅ Return final inputs
        return {key: val.squeeze(0) if isinstance(val, torch.Tensor) and key != "pixel_values" else val for key, val in inputs.items()}


print(f"pixel_values dtype: {pixel_values.dtype}")
print(f"labels dtype: {labels.dtype if 'labels' in locals() else 'No labels'}")


pixel_values dtype: torch.float32
labels dtype: torch.int64


In [6]:
# Load Model and Processor
print("🔄 Loading model and processor from cache...")
processor = AutoProcessor.from_pretrained(MODEL_NAME, local_files_only=True)
model = AutoModelForImageTextToText.from_pretrained(MODEL_NAME, local_files_only=True)
print("✅ Model and processor loaded.")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🔄 Loading model and processor from cache...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model and processor loaded.


In [13]:
def collate_fn(batch):
    # ✅ Extract individual items from the batch
    pixel_values = torch.stack([item["pixel_values"] for item in batch])  # Batch pixel values
    image_grid_thw = torch.stack([item["image_grid_thw"] for item in batch])  # Batch image grid

    # ✅ Handle variable-length labels using padding
    labels = [item["labels"] for item in batch if "labels" in item]
    if labels:
        # Use pad_sequence to pad labels to the longest sequence in the batch
        from torch.nn.utils.rnn import pad_sequence
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # Use -100 for ignored tokens
    else:
        labels = None

    # ✅ Combine into a dictionary
    batch_dict = {
        "pixel_values": pixel_values,
        "image_grid_thw": image_grid_thw,
    }

    if labels is not None:
        batch_dict["labels"] = labels

    return batch_dict


In [21]:
# ✅ Load full dataset
train_dataset = ColorImageDataset("color_images_metadata.json", processor)

# ✅ Split into training and validation datasets (80/20 split)
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"✅ Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

# ✅ Ensure DataLoader maintains batch dimension (Training)
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn  # Use custom collate_fn
)

# ✅ Ensure DataLoader maintains batch dimension (Validation)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,  # No need to shuffle validation data
    collate_fn=collate_fn
)

# ✅ Debug: Check one batch from training and validation
for batch in train_loader:
    print(f"🔥 Train Batch pixel_values shape: {batch['pixel_values'].shape}")
    print(f"🔥 Train Batch labels shape: {batch['labels'].shape if 'labels' in batch else 'No labels'}")
    break

for batch in val_loader:
    print(f"🔥 Validation Batch pixel_values shape: {batch['pixel_values'].shape}")
    print(f"🔥 Validation Batch labels shape: {batch['labels'].shape if 'labels' in batch else 'No labels'}")
    break


🔄 Loading dataset...
✅ Dataset loaded with 200 QA pairs.
✅ Dataset split: 160 training samples, 40 validation samples.
🔍 Loading image from: color_img/color_img_94.png
🔍 Loading pixel values from: pixel_values/pixel_values_94.json
📝 Description: This is a solid white color image.
❓ Question: Is this color some other color?
💬 Answer: No
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])
🔍 Loading image from: color_img/color_img_18.png
🔍 Loading pixel values from: pixel_values/pixel_values_18.json
📝 Description: This is a solid brown color image.
❓ Question: Is this color brown?
💬 Answer: Yes
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.

In [22]:
# Training Configuration
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_dir=LOG_DIR,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=SAVE_LIMIT,
    push_to_hub=PUSH_TO_HUB,
    fp16=USE_FP16,
    report_to="none"
)

print("Training configurations set")

Training configurations set


/home/capstone_student/Documents/sinarmas_project_flowchart/venv/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# Check if label values exceed vocab size
vocab_size = tokenizer.vocab_size
print("Tokenizer vocab size:", vocab_size)

# Debug the first batch of labels
for i, batch in enumerate(train_dataset):
    labels = batch["labels"]
    print(f"\n🔢 Entry {i + 1} - Labels as Token IDs:")
    print(labels)

    # Ensure labels are within vocab range
    if (labels >= vocab_size).any():
        print(f"❗ Warning: Labels exceed vocab size in entry {i + 1}")
        break


Tokenizer vocab size: 151643
🔍 Loading image from: color_img/color_img_31.png
🔍 Loading pixel values from: pixel_values/pixel_values_31.json
📝 Description: This is a solid black color image.
❓ Question: Is this color black?
💬 Answer: Yes
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])

🔢 Entry 1 - Labels as Token IDs:
tensor([9454])
🔍 Loading image from: color_img/color_img_24.png
🔍 Loading pixel values from: pixel_values/pixel_values_24.json
📝 Description: This is a solid pink color image.
❓ Question: Is this color pink?
💬 Answer: Yes
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])

🔢 Entry 2 - Labels as Token I

In [24]:
# Initialize Trainer
print("🔄 Initializing trainer...")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.tokenizer
)

print("Trainer Initalized")

🔄 Initializing trainer...


/tmp/ipykernel_257495/2149996427.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2025-02-25 13:39:58,604] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Trainer Initalized


/home/capstone_student/miniconda3/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/capstone_student/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/capstone_student/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/capstone_student/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/capstone_student/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/capstone_student/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/capstone_studen

In [25]:
# Start Fine-Tuning
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

# Save Fine-Tuned Model
print(f"💾 Saving fine-tuned model to {OUTPUT_DIR}...")
model.save_pretrained(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved successfully in: {OUTPUT_DIR}")

🚀 Starting training...
🔍 Loading image from: color_img/color_img_53.png
🔍 Loading pixel values from: pixel_values/pixel_values_53.json
📝 Description: This is a solid yellow color image.
❓ Question: Is this color some other color?
💬 Answer: No
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])
🔍 Loading image from: color_img/color_img_64.png
🔍 Loading pixel values from: pixel_values/pixel_values_64.json
📝 Description: This is a solid white color image.
❓ Question: Is this color white?
💬 Answer: Yes
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])


RuntimeError: shape '[8, 2, 1, 2]' is invalid for input of size 48

In [26]:
for batch in train_loader:
    print(f"🔥 pixel_values shape before forward pass: {batch['pixel_values'].shape}")

    # 🚨 Debugging forward pass
    try:
        outputs = model(**batch)
    except RuntimeError as e:
        print("🚨 Error during forward pass!")
        print("🔥 pixel_values shape:", batch["pixel_values"].shape)
        print("🔥 labels shape:", batch["labels"].shape)
        raise e


🔍 Loading image from: color_img/color_img_64.png
🔍 Loading pixel values from: pixel_values/pixel_values_64.json
📝 Description: This is a solid white color image.
❓ Question: Is this color white?
💬 Answer: Yes
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])
🔍 Loading image from: color_img/color_img_31.png
🔍 Loading pixel values from: pixel_values/pixel_values_31.json
📝 Description: This is a solid black color image.
❓ Question: Is this color some other color?
💬 Answer: No
📦 Processor output keys: dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
🏷️ Final labels dtype: torch.int64
🏷️ labels tensor shape: torch.Size([1, 1])
📏 pixel_values tensor shape: torch.Size([1, 3, 224, 224])
🔍 Loading image from: color_img/color_img_93.png
🔍 Loading pixel values from: pixel_values/pixel

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not NoneType

In [ ]:
# Forward pass debug
try:
    outputs = model(**batch)
except RuntimeError as e:
    print("🚨 Error during forward pass!")
    print("pixel_values shape:", batch["pixel_values"].shape)
    print("labels shape:", batch["labels"].shape if "labels" in batch else "No labels")
    raise e
